<a href="https://colab.research.google.com/github/timbosssds/ML/blob/main/Chatbot_with_citations_wip_convomemory_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
# !pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers
# !pip install pypdf
# !pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 5.9 MB/s eta 0:0

In [ ]:
# MileStone 1
# -- end-to-end (No Citations)

# -- 1.0 Load the PDF document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
#PDF_PATH = open(r'/content/cba_fsg.pdf', mode='rb')
PDF_PATH = '/content/cba_fsg.pdf'
loader = PyPDF2.PdfReader(PDF_PATH)

# -- 2.0 Create an empty string to store the text
text = ""
# Loop through each page of the PDF
for page_num in range(len(loader.pages)):
    # Get the page object
    page = loader.pages[page_num]
    # Extract the texst from the page and add it to the text variable
    text += page.extract_text()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)

# -- 3.0 Process text
def convert_list_to_string_with_speech_marks(list_items):
  """Converts a list to a string with speech marks at each end."""
  string = " ".join(list_items)
  string = f"\"{string}\""
  return string
list_items = texts
text = convert_list_to_string_with_speech_marks(list_items)

# -- Model
context=text
from transformers import pipeline
def generate_answer(question, context):
  """Generates an answer to a question given a context."""
  question_answerer = pipeline("question-answering", model="deepset/roberta-base-squad2")
  response = question_answerer(question=question, context=text, max_answer_len=1024, top_p=1)
  response = response[list(response)[-1]]
  return response

# -- Call model and produce answer
# Generate an answer to the question
#question = "what is the Eligibility criteria for the Product"
question = "what is the date"

answer = generate_answer(question, context)
print(answer)

1 October 2022


In [ ]:
# MileStone 2
# -- end-to-end (With Citations)
# -- 1.0 Load the PDF document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
#PDF_PATH = open(r'/content/cba_fsg.pdf', mode='rb')
PDF_PATH = '/content/cba_fsg.pdf'
loader = PyPDF2.PdfReader(PDF_PATH)

# -- 2.0 Create an empty string to store the text
text = ""
# Loop through each page of the PDF
for page_num in range(len(loader.pages)):
    # Get the page object
    page = loader.pages[page_num]
    # Extract the texst from the page and add it to the text variable
    text += page.extract_text()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)

# -- 3.0 Process text
def convert_list_to_string_with_speech_marks(list_items):
  """Converts a list to a string with speech marks at each end."""
  string = " ".join(list_items)
  string = f"\"{string}\""
  return string
list_items = texts
text = convert_list_to_string_with_speech_marks(list_items)

# -- Model
context=text
# Define a function to generate an answer to a question with citations
from transformers import pipeline

def generate_answer_with_citations(question, context):
    """Generates an answer to a question given a context and provides citations."""
    question_answerer = pipeline("question-answering", model="deepset/roberta-base-squad2")
    response = question_answerer(question=question, context=context, max_answer_len=1024, top_p=1)
    answer_text = response[list(response)[-1]]
    answer_start = response["start"]
    answer_end = response["end"]

    # Get the passage containing the answer
    start_index = max(0, context.rfind(".", 0, answer_start))
    end_index = min(len(context), context.find(".", answer_end))
    passage_with_answer = context[start_index:end_index].strip()

    # Get the citation (page number) of the passage
    page_number = context.count(".", 0, start_index) + 1

    return {
        "answer": answer_text,
        "passage": passage_with_answer,
        "citation": f"Page {page_number}"
    }

# Example usage
#question = "what is the date"
question = "what commision do you receive"
answer_info = generate_answer_with_citations(question, context)
print("Answer:", answer_info["answer"])
print("Passage:", answer_info["passage"])
print("Citation:", answer_info["citation"])

Answer: via email, by telephone or fax
Passage: . 
However, there are special arrangements in place for some 
products and services where we can receive your instructions 
electronically via email, by telephone or fax
Citation: Page 13


In [ ]:
# -- All below is WIP...

In [6]:
# WIP - trying to get conversation history
# -- end-to-end (With Citations)
# -- 1.0 Load the PDF document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
#PDF_PATH = open(r'/content/cba_fsg.pdf', mode='rb')
PDF_PATH = '/content/cba_fsg.pdf'
loader = PyPDF2.PdfReader(PDF_PATH)

# -- 2.0 Create an empty string to store the text
text = ""
# Loop through each page of the PDF
for page_num in range(len(loader.pages)):
  # Get the page object
  page = loader.pages[page_num]
  # Extract the texst from the page and add it to the text variable
  text += page.extract_text()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)

# -- 3.0 Process text
def convert_list_to_string_with_speech_marks(list_items):
  """Converts a list to a string with speech marks at each end."""
  string = " ".join(list_items)
  string = f"\"{string}\""
  return string
  list_items = texts
  text = convert_list_to_string_with_speech_marks(list_items)

# -- 4.0 Maintain conversation history
conversation_history = []

# -- Model
context=text

# Define a function to generate an answer to a question with citations
from transformers import pipeline

def generate_answer_with_citations(question, context):
  """Generates an answer to a question given a context and provides citations."""
  # Check if the question is related to any of the previous questions
  for previous_question, previous_answer in conversation_history:
    if question.startswith(previous_question):
      context = f"{context} {previous_answer}"

  question_answerer = pipeline("question-answering", model="deepset/roberta-base-squad2")
  response = question_answerer(question=question, context=context, max_answer_len=1024, top_p=1)
  answer_text = response[list(response)[-1]]
  answer_start = response["start"]
  answer_end = response["end"]

  # Get the passage containing the answer
  start_index = max(0, context.rfind(".", 0, answer_start))
  end_index = min(len(context), context.find(".", answer_end))
  passage_with_answer = context[start_index:end_index].strip()

  # Get the citation (page number) of the passage
  page_number = context.count(".", 0, start_index) + 1

  # Add the new question and answer to the conversation history
  conversation_history.append((question, answer_text))

  return {
      "answer": answer_text,
      "passage": passage_with_answer,
      "citation": f"Page {page_number}"}

# Example usage
#question = "what is the date" # original question

#question = "what is afca?" # initial q to test follow up question
question = "what number do i contact you on?" # follow up question to see if conversation memory working

#question = "what email do i use for dispute resolution?" # Intial question
#question = "and phone number?" # Intial question # Follow up

answer_info = generate_answer_with_citations(question, context)
print("Answer:", answer_info["answer"])
print("Passage:", answer_info["passage"])
print("Citation:", answer_info["citation"])


Answer: 1300 555 727
Passage: .au/feedback
National Relay Service 
• TTY or Voice: Call 133 677  then as k for 13 2221
• Speak & listen: Call 1300 555 727  then as k for 13 2221
• SMS relay – Text  0423 677 767
For more information, visit the National Relay Service: 
relayservice
Citation: Page 43


In [ ]:
conversation_history

[('what number do i contact you on?', '1300 555 727')]

In [15]:
# -- WIP (trying to use langchain memory)
# MileStone 2
# -- end-to-end (With Citations)
# -- 1.0 Load the PDF document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
#PDF_PATH = open(r'/content/cba_fsg.pdf', mode='rb')
#PDF_PATH = '/content/cba_fsg.pdf'
PDF_PATH = '/content/bendigo-term-deposit-accounts-facilities.pdf'
loader = PyPDF2.PdfReader(PDF_PATH)


# -- New (add chat memory)
# https://python.langchain.com/docs/modules/memory/
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# -- New (add chat memory)
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()



# -- 2.0 Create an empty string to store the text
text = ""
# Loop through each page of the PDF
for page_num in range(len(loader.pages)):
    # Get the page object
    page = loader.pages[page_num]
    # Extract the texst from the page and add it to the text variable
    text += page.extract_text()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)

# -- 3.0 Process text
def convert_list_to_string_with_speech_marks(list_items):
  """Converts a list to a string with speech marks at each end."""
  string = " ".join(list_items)
  string = f"\"{string}\""
  return string
list_items = texts
text = convert_list_to_string_with_speech_marks(list_items)

# -- Model
context=text
# Define a function to generate an answer to a question with citations
from transformers import pipeline

def generate_answer_with_citations(question, context): #, memory
    """Generates an answer to a question given a context and provides citations."""
    question_answerer = pipeline("question-answering", model="deepset/roberta-base-squad2")
    response = question_answerer(question=question, context=context, max_answer_len=1024, top_p=1)
    answer_text = response[list(response)[-1]]
    answer_start = response["start"]
    answer_end = response["end"]

    # -- New (add chat memory)
    #MessagesPlaceholder(variable_name="chat_history")

    # Get the passage containing the answer
    start_index = max(0, context.rfind(".", 0, answer_start))
    end_index = min(len(context), context.find(".", answer_end))
    passage_with_answer = context[start_index:end_index].strip()

    # Get the citation (page number) of the passage
    page_number = context.count(".", 0, start_index) + 1

    return {
        "answer": answer_text,
        "passage": passage_with_answer,
        "citation": f"Page {page_number}"
    }

# -- New (add chat memory)
#memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Example usage
#question = "what number do i contact you on?" # follow up question to see if conversation memory working
question = "what isthe initial deposit?"
#question = "what email do i use for dispute resolution?" # Intial question
#question = "and phone number?" # Intial question # Follow up
#answer_info = generate_answer_with_citations(question, context, memory)

answer_info = generate_answer_with_citations(question, context)
print("Answer:", answer_info["answer"])
print("Passage:", answer_info["passage"])
print("Citation:", answer_info["citation"])

Answer: the amount you initially deposit with us 
when you open your account
Passage: .  
 
"initial deposit"  means the amount you initially deposit with us 
when you open your account
Citation: Page 65


In [12]:
#def generate_answer_with_citations(question, context): #, memory
"""Generates an answer to a question given a context and provides citations."""
question_answerer = pipeline("question-answering", model="deepset/roberta-base-squad2")
response = question_answerer(question=question, context=context, max_answer_len=1024, top_p=1)
answer_text = response[list(response)[-1]]
answer_start = response["start"]
answer_end = response["end"]

# -- New (add chat memory)
#MessagesPlaceholder(variable_name="chat_history")

# Get the passage containing the answer
start_index = max(0, context.rfind(".", 0, answer_start))
end_index = min(len(context), context.find(".", answer_end))
passage_with_answer = context[start_index:end_index].strip()

# Get the citation (page number) of the passage
page_number = context.count(".", 0, start_index) + 1
print(page_number)

    # return {
    #     "answer": answer_text,
    #     "passage": passage_with_answer,
    #     "citation": f"Page {page_number}"
    # }

65


In [77]:
## Part of embedding - can i make the data to df, then link have text in one col and embeddings in next

# Need to get embeddings going...
# Bard: python open source code to embed pdf doc as vectors in dataframe

# Not sure below required.
# import faiss
# import numpy as np
# import pandas as pd
# import sentence_transformers


# -- 1.0 Load the PDF document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
#PDF_PATH = open(r'/content/cba_fsg.pdf', mode='rb')
#PDF_PATH = '/content/cba_fsg.pdf'
PDF_PATH = '/content/bendigo-term-deposit-accounts-facilities.pdf'
loader = PyPDF2.PdfReader(PDF_PATH)


# -- New (add chat memory)
# https://python.langchain.com/docs/modules/memory/
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# -- New (add chat memory)
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()


# -- 2.0 Create an empty string to store the text + now store the text in df.
# Once text added to df, call this to add new col with text embeddings.
import gensim
model = gensim.models.Word2Vec(chunk, min_count=1)
embeddings = model.wv.vectors

text = ""
# Loop through each page of the PDF
for page_num in range(len(loader.pages)):
    # Get the page object
    page = loader.pages[page_num]
    # Extract the texst from the page and add it to the text variable
    text += page.extract_text()

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(text)

# Tokenize each chunk
tokenized_texts = [text.split() for text in texts]

# Train Word2Vec model
model = gensim.models.Word2Vec(tokenized_texts, min_count=1)

# Function to get embeddings for each word in a chunk
def get_embeddings(chunk):
    return [model.wv[word] if word in model.wv else None for word in chunk.split()]

# Create DataFrame with chunks and their embeddings
df = pd.DataFrame(texts, columns=['chunk'])
df['embeddings'] = df['chunk'].apply(get_embeddings)

# Display DataFrame
display(df)
print(df.shape)



# -- Need to embed question
question = "what is the initial deposit?"
def generate_embeddings(text):
  """Generates embeddings using the Gensim library."""
  model = gensim.models.Word2Vec(text, min_count=1)
  embeddings = model.wv.vectors
  return embeddings
q = generate_embeddings(question)
dfq = pd.DataFrame([[q]], columns=['data'])
dfq['chunk'] = question
dfq = dfq[['chunk', 'data']]
display(dfq)







,chunk,embeddings
0,Bendigo Term Deposit Accounts and Facilities -...,"[[-0.08560844, 0.148259, 0.048547424, -0.07217..."
1,The issuer of the products described in this d...,"[[-0.063090034, 0.0969673, 0.033387315, -0.032..."
2,determine the rate of interest we pay on your ...,"[[0.0026926412, 0.0012637266, -0.0021947091, -..."
3,Additional Deposits Within the first 7 days o...,"[[0.0001396415, 0.014425261, -0.0045510964, 0...."
4,The terms and conditions beginning on page 6 e...,"[[-0.063090034, 0.0969673, 0.033387315, -0.032..."
...,...,...
63,"25.1 We can decide, at our discretion, the ord...","[[0.0020451718, 0.008724767, 0.006376833, 0.00..."
64,(e) if you are registered for Bendigo e -banki...,"[[-0.0056517087, 0.015433809, -0.0027306234, -..."
65,debt. Any reasonable expenses we incur in reco...,"[[-0.0018903115, -0.00023423944, -0.009434234,..."
66,conditions that is illegal or unenforceable wi...,"[[-0.042209182, 0.079314075, 0.013568966, -0.0..."


(68, 2)


,chunk,data
0,what is the initial deposit?,"[[-0.00053622725, 0.00023643136, 0.0051033497,..."


In [65]:
# -- Search df to find best context
import numpy as np
target_row = dfq.iloc[0]
def cosine_similarity(row1, row2):
    # Calculate the dot product of the two vectors
    dot_product = np.dot(row1, row2)
    # Calculate the norm of each vector
    norm1 = np.linalg.norm(row1)
    norm2 = np.linalg.norm(row2)
    # Calculate the cosine similarity
    cosine_similarity = dot_product / (norm1 * norm2)
    return cosine_similarity

similarity_scores = []
for index, row in df.iterrows():
    similarity_score = cosine_similarity(row, target_row)
    similarity_scores.append(similarity_score)
similarity_df = pd.DataFrame(similarity_scores, columns=['similarity_score'])
top_n_df = similarity_df.sort_values(by='similarity_score', ascending=False).head(n)
top_n_similar_rows = df1.merge(top_n_df, on='index')
# -- The pass both to model

TypeError: ignored